In [1]:
# Install prerequisite libraries
%%capture
!pip install world_bank_data
!pip install -U plotly

In [2]:
# data management
import pandas as pd
import plotly.io as pio

# data visualization
import plotly
import plotly.offline as offline
import plotly.graph_objects as go

# data capture
import world_bank_data as wb

offline.init_notebook_mode(connected = True)

In [3]:
# Countries and associated regions
countries = wb.get_countries()

# Get world population data
population = wb.get_series('SP.POP.TOTL', id_or_value = 'id', simplify_index = True, mrv = 1)

# Aggregate region, country and population
world_pop = countries[['region', 'name']].rename(columns={'name': 'country'}).loc[countries.region != 'Aggregates']
world_pop['population'] = population

In [13]:
# The sunburst plot requires weights (values), labels, and parent (region, or World)
# We build the corresponding table here
columns = ['parents', 'labels', 'values']

level1 = world_pop.copy()
level1.columns = columns
level1['text'] = level1['values'].apply(lambda pop: f'{pop:,.0f}')


level2 = world_pop.groupby('region').population.sum().reset_index()[['region', 'region', 'population']]
level2.columns = columns
level2['parents'] = 'World'
# move value to text for this level
level2['text'] = level2['values'].apply(lambda pop: f'{pop:,.0f}')
level2['values'] = 0

level3 = pd.DataFrame({'parents': [''],
                       'labels': ['World'],
                       'values': [0.0],
                       'text': [f"{population.loc['WLD']:,.0f}"]
                      })

all_levels = pd.concat([level1, level2, level3], axis=0).reset_index(drop=True)
all_levels.head()

,parents,labels,values,text
0,Latin America & Caribbean,Aruba,106445.0,"106,445"
1,South Asia,Afghanistan,41128771.0,"41,128,771"
2,Sub-Saharan Africa,Angola,35588987.0,"35,588,987"
3,Europe & Central Asia,Albania,2775634.0,"2,775,634"
4,Europe & Central Asia,Andorra,79824.0,"79,824"


In [88]:
from plotly.validators.layout import Paper_BgcolorValidator
# set custom colours

seq_1 = ['#a6cee3','#1f78b4','#b2df8a',
         '#33a02c','#fb9a99','#e31a1c',
         '#fdbf6f','#ff7f00','#cab2d6',
         '#6a3d9a']

# naming a layout theme for future reference
pio.templates["custom"] = go.layout.Template(
    layout_colorway = seq_1
)

# setting Google color palette as default
pio.templates.default = "custom"

# plot world population

label_font_size = 12
label_font_fam = "Montserrat, sans-serif"

fig = go.Figure(go.Sunburst(
    ids = all_levels['labels'],
    labels = all_levels['labels'],
    parents = all_levels['parents'],
    values = all_levels['values'],
    hovertext = all_levels['text'],
    textfont = dict(size = label_font_size, family = label_font_fam)
))

# Set the hovertemplate to only show the value (population) without the label (region name)
fig.update_traces(hovertemplate = '<b>%{label}</b><br>Pop_2022: %{hovertext}')

# Disable the display of 'trace 0' when hovering over a region
fig.update_layout(hoverlabel = dict(
    bgcolor ="white", font_size = 12, font_family = label_font_fam))

# Set font family
title_font_family = "Roboto, sans-serif"

# Set font size for the title
title_font_size = 24

fig.update_layout(title = 'World Population (World Bank, 2022)',
                  autosize = True,
                  height = 800,
                  width = 800,
                  title_x = 0.5,
                  title_font = dict(size = title_font_size, family = title_font_family),
                  # paper_bgcolor = "#d9d9d9"
                  )
fig.add_annotation(dict(
    font = dict(color = 'black',
                size = 15,
                family = title_font_family),
    x = 0.5,
    y = -0.1,
    xref = "paper",
    yref = "paper",
    showarrow = False,
    text = 'NB: Explore population dynamics by clicking on a region to gain deeper insights.'
))

# Save the plot as an HTML file
fig.write_html('world_population_sunburst.html', auto_open = True)
# pio.write_html(fig, default_width = '50%', file = 'world_pop_2022.html')

# Display the plot
fig.show(renderer = "colab")

In [ ]:
# download final output
from google.colab import files

files.download('world_population_sunburst.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>